In [7]:
import re
import pandas as pd
from Bio import Entrez

In [ ]:
Entrez.email = 'chen1i6c04@gmail.com'

def einfo(db):
    with Entrez.einfo(db=db) as handle:
        record = Entrez.read(handle)
    for fileld in record['DbInfo']['FieldList']:
        print(fileld['Name'], fileld['Description'])

def egquery(term, db='sra'):
    with Entrez.egquery(term=term) as handle:
        record = Entrez.read(handle)
    for row in record["eGQueryResult"]:
        if row["DbName"]==db:
            count = int(row["Count"])
            return count

def esearch(term, db="sra", retmax=10000):
    with Entrez.esearch(db=db, term=term, retmax=retmax) as handle:
        record = Entrez.read(handle)
        return record['IdList']

def efetch(uid, db, rettype='text', retmode="text"):
    with Entrez.efetch(db=db, id=uid, rettype=rettype, retmode=retmode) as handle:
        record = handle.read()
        return record

In [ ]:
database = 'biosample'

In [ ]:
einfo(database)

In [ ]:
sample_info = pd.read_csv('/media/Synology_222/2_Research/03_cgMLST/Enterobacter_hormaechei/Enterobacter_hormaechei_SampleInfo.csv', usecols=['BioSample'])

In [ ]:
accessions = sample_info['BioSample'].drop_duplicates().to_list()

In [ ]:
batch_size = 500

groups = []
for start in range(0, len(accessions), batch_size):
    end = start + batch_size
    group = accessions[start: end]
    groups.append(group)

In [ ]:
results = ''
for group in groups:
    patterm = ' OR '.join(f'{accession}[ACCN]' for accession in group)
    uids = esearch(patterm, database)
    result = efetch(uids, database, 'text', 'text')
    results += result
    sleep(10)

In [ ]:
sample_info = []
entrys = results.strip().split('\n\n')
for entry in entrys:
    sample_info.append(SampleInfoScanner().parse_entry(entry))

In [34]:
def entryIterator(handle):
    term = ''
    for line in f:
        if line.strip() == '' and term != '':
            yield term
            term = ''
            continue
        term += line

In [49]:
useless_term = ['None detected', 'not applicable', 'not data', 'not available', 'Not applicable',
                'Unknown', 'not typed', 'unknown', 'not provided', 'not_checked', 'Not collected',
                'Not Applicable', 'Not provided', 'None', 'Missing', '?', 'missing', 'NA', 'n/a',
                'Not available', 'not collected']

In [53]:
filepath = '/home/chen1i6c04/下載/biosample_result.txt'

In [54]:
enteries = []
with open(filepath) as f:
    for entry in entryIterator(f):
        attribute = dict(re.findall('/(.*)="(.*)"', entry))
        accession = re.search('Accession: ([A-Z]+[0-9]+)', entry).groups()[0]
        attribute['BioSample'] = accession
        enteries.append(attribute)

In [55]:
len(enteries)

447116

In [57]:
df = pd.DataFrame(enteries[8000:9000]).replace(useless_term, pd.NA).dropna(axis=1, how='all').dropna(axis=0, how='all')

In [58]:
df

,strain,collection date,geographic location,isolation and growth condition,latitude and longitude,number of replicons,BioSample,isolate,serovar,isolate name alias,...,ENA-LAST-UPDATE,IFSAC+ Category,source type,PublicAccession,attribute_package,ProjectAccession,Genus,ontological term,FDA_Lab_Id,Species
0,SZL 30,<NA>,<NA>,foodstuffs,<NA>,3,SAMN22551430,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SZL 31,<NA>,<NA>,foodstuffs,<NA>,3,SAMN22551429,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PNUSAS250773,2021,<NA>,NaN,<NA>,NaN,SAMN24218467,<NA>,Newport,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PNUSAS250771,2021,<NA>,NaN,<NA>,NaN,SAMN24218466,<NA>,Typhi,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PNUSAS250772,2021,<NA>,NaN,<NA>,NaN,SAMN24218465,<NA>,Needs further review,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,PNUSAS250248,<NA>,USA,NaN,<NA>,NaN,SAMN24040633,<NA>,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,PNUSAS250241,<NA>,USA,NaN,<NA>,NaN,SAMN24040629,<NA>,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,PNUSAS250246,<NA>,USA,NaN,<NA>,NaN,SAMN24040626,<NA>,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,PNUSAS250247,<NA>,USA,NaN,<NA>,NaN,SAMN24040623,<NA>,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
